In [113]:
test = """        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5"""
source = test

In [114]:
source = open("./day_22.txt").read()

## Step 0: Parse the source data

In [115]:
(board, instructions) = source.split("\n\n")

In [116]:
# Parse instructions
steps = []
digits = []
for char in instructions:
    if char in "LR":
        if(len(digits)): steps.append({"type": "move", "steps": int("".join(digits))})
        # instructions = instructions.replace(char, f" {char} ")
        steps.append({"type": "turn", "direction": char})
        digits = []
    else: digits.append(char)
if(len(digits)): steps.append({"type": "move", "steps": int("".join(digits))})


In [117]:
WARP = '='
WALL = '|'
SPACE = "_"

In [118]:
board = board.replace("#", WALL)
boardRows = [WARP + line.replace(" ", WARP) + WARP for line in  board.splitlines()]
boardRows = [row.replace('.', SPACE) for row in boardRows]
maxlen = max([len(row) for row in boardRows])
boardRows = ["".join(WARP for _ in range(maxlen)), *[row + "".join(WARP for _ in range(maxlen - len(row))) for row in boardRows], "".join(WARP for _ in range(maxlen))]
boardGrid = [list(row) for row in boardRows]


In [119]:
def viz_grid(boardGrid, position = [0,0], spacer = "  "):
    newArr = boardGrid.copy()
    newArr[position[1]][position[0]] = 'R'
    return "\n".join([spacer.join(row) for row in newArr])

In [120]:
import numpy

boardArr = numpy.array(boardGrid)
startRow = 1
startCol =  boardArr[startRow].tolist().index(SPACE)

In [121]:
def runStep(step, facing, position, grid, stepIndex = 0, isDebug = False):
    if isDebug: print("Starting", step, facing, position, stepIndex)
    (col, row) = position
    if step["type"] == "turn":
        if step["direction"] == "R":
            facing = (facing - 1) % 4
            grid = numpy.rot90(grid, 1, axes=(0, 1))
            c,r = position
            row = len(grid) - c-1
            col = r
        else:
            facing = (facing + 1) % 4
            grid = numpy.rot90(grid, 3, axes=(0, 1))
            c,r = position
            col = len(grid[0]) - r-1
            row = c
    else:
        steps = step["steps"]
        c = col
        r = grid[row]
        r2 = r.copy()
        r2[col] = "R"
        if isDebug: print(''.join(r2))
        l = len([c for c in r if c != WARP])
        offset = next(i for i in range(len(r)) if r[i] != WARP)
        wallpos = next(i for i in range(col, len(r)) if r[i] == WALL) if WALL in r[col:] else -1
        prevwall = next(i for i in range(offset, col) if r[i] == WALL) if col > offset and WALL in r[:col] else -1
        if wallpos > -1 and steps > wallpos - col-1:
            col = wallpos - 1
        elif col + steps < (wallpos if wallpos > -1 else (offset + l)):
            col = (((col-offset) + steps) ) + offset
        elif (prevwall > -1) and (((l-(col-offset)) + (prevwall - offset))<= steps):
            if(prevwall > offset): col = prevwall -1
            else: col = offset +l -1
        else:
            col = (((col-offset) + steps) % l) + offset
        if isDebug: 
            r2[col] = "S"
            print(''.join(r2))
        r2[col] = "S"
        if(r[col] != SPACE): 
            print("BADMANMBADMANBADMAN", r[col], step, (c,r), stepIndex)
            return None

    return (facing, (col, row), grid)

## Part 1: Wrapping Map

In [122]:
position = (startCol, startRow)
grid = boardArr.copy()
facing = 0

In [123]:
from time import sleep
from IPython.display import clear_output
for i, step in enumerate(steps): facing, position, grid = runStep(step, facing, position, grid, i)

In [124]:
originalFacing = facing
while facing > 0:
    print("Rotating because facing is ", facing)
    facing, position, grid = runStep({"type": "turn", "direction": 'R'}, facing, position, grid)

1000 * position[1] + 4 * position[0] + originalFacing

75388